In [1]:
!pip install -q torch datasets transformers scikit-learn accelerate pandas


In [2]:
!pip install -U "transformers>=4.40" "datasets>=2.19" "accelerate>=0.30"


In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          Trainer, TrainingArguments)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# -------- CONFIG --------
BASE_MODEL_DIR   = "../outputs/initial/model"       # continue from initial
OUTPUT_DIR       = "../outputs/with_pseudo"
MODEL_OUT        = os.path.join(OUTPUT_DIR, "model")

LABELED_PATHS    = ["../data/seed_labels.csv", "../data/expanded_seed.csv", "../data/uncertain_labels.csv"]
PSEUDO_PATH      = "../data/pseudo_labels.csv"

TEST_SIZE        = 0.15
SEED             = 42
MAX_LENGTH       = 128
BATCH_SIZE       = 25
EPOCHS           = 4
LR               = 2e-5

# Balance: cap pseudo to ratio of gold labels (e.g., 1.0 => 1:1)
PSEUDO_TO_GOLD_RATIO = 1.0
# Or set fixed cap (overrides ratio if not None)
PSEUDO_CAP           = None  # e.g., 300
# Stratified per-class cap helper will keep classes balanced
# -----------------------------------------

torch.manual_seed(SEED)
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [4]:
# Load gold (human) labels
gold_dfs = []
for p in LABELED_PATHS:
    if os.path.exists(p):
        df = pd.read_csv(p)
        if set(["text","label"]).issubset(df.columns):
            gold_dfs.append(df[["text","label"]])
        else:
            print(f"Skipping {p}: missing columns.")
    else:
        print(f"Not found: {p}")

if not gold_dfs:
    raise ValueError("No gold labeled data found.")

gold = pd.concat(gold_dfs, ignore_index=True).dropna().copy()
gold["text"] = gold["text"].astype(str)
gold["label"] = gold["label"].astype(int)
gold = gold.drop_duplicates(subset="text").reset_index(drop=True)

print("Gold count:", len(gold), "class counts:", gold["label"].value_counts().to_dict())

# Load pseudo
if not os.path.exists(PSEUDO_PATH):
    raise ValueError("pseudo_labels.csv not found. Run pseudo_label.ipynb first.")
pseudo = pd.read_csv(PSEUDO_PATH).dropna().copy()
pseudo["text"] = pseudo["text"].astype(str)
pseudo["label"] = pseudo["label"].astype(int)
pseudo = pseudo.drop_duplicates(subset="text").reset_index(drop=True)
print("Pseudo total:", len(pseudo), "class counts:", pseudo["label"].value_counts().to_dict())

# Balance pseudo size
def stratified_cap(df, per_class):
    return df.groupby("label", group_keys=False).apply(
        lambda g: g.sample(min(len(g), per_class), random_state=SEED)
    )

target_pseudo = len(gold) if PSEUDO_TO_GOLD_RATIO is None else int(len(gold) * PSEUDO_TO_GOLD_RATIO)
if PSEUDO_CAP is not None:
    target_pseudo = min(target_pseudo, PSEUDO_CAP)

# Per-class cap (split across two classes)
per_class_cap = max(1, target_pseudo // 2)
pseudo_bal = stratified_cap(pseudo, per_class=per_class_cap)

print("Pseudo kept:", len(pseudo_bal), "class counts:", pseudo_bal["label"].value_counts().to_dict())

# Build final train DF
train_all = pd.concat([gold, pseudo_bal], ignore_index=True).drop_duplicates("text").reset_index(drop=True)
print("Final train size:", len(train_all), "class counts:", train_all["label"].value_counts().to_dict())

# Split into train/val
train_df, val_df = train_test_split(
    train_all, test_size=TEST_SIZE, random_state=SEED, stratify=train_all["label"]
)
print("Train size:", len(train_df), "Val size:", len(val_df))


Gold count: 398 class counts: {0: 199, 1: 199}
Pseudo total: 50 class counts: {1: 50}
Pseudo kept: 50 class counts: {1: 50}
Final train size: 448 class counts: {1: 249, 0: 199}
Train size: 380 Val size: 68


C:\Users\Hp\AppData\Local\Temp\ipykernel_57104\3588539374.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("label", group_keys=False).apply(


In [5]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_DIR)

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH
    )

train_ds = Dataset.from_pandas(train_df)
val_ds   = Dataset.from_pandas(val_df)

train_ds = train_ds.map(tokenize_batch, batched=True)
val_ds   = val_ds.map(tokenize_batch,   batched=True)

train_ds = train_ds.remove_columns([c for c in train_ds.column_names if c not in ("input_ids","attention_mask","label")])
val_ds   = val_ds.remove_columns(  [c for c in val_ds.column_names   if c not in ("input_ids","attention_mask","label")])

train_ds = train_ds.with_format("torch")
val_ds   = val_ds.with_format("torch")


Map:   0%|          | 0/380 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

In [6]:
id2label = {0: "non_hate", 1: "hate"}
label2id = {"non_hate": 0, "hate": 1}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    acc  = accuracy_score(labels, preds)
    f1   = f1_score(labels, preds, average="binary", pos_label=1)
    prec = precision_score(labels, preds, average="binary", pos_label=1)
    rec  = recall_score(labels, preds, average="binary", pos_label=1)
    return {"accuracy": acc, "f1": f1, "precision": prec, "recall": rec}

# Class weights on the *train* mix
counts = train_df["label"].value_counts()
cw = torch.tensor([1.0, 1.0])
for cls in [0,1]:
    if cls in counts:
        cw[cls] = len(train_df) / (2.0 * counts[cls])
cw = cw / cw.mean()

from transformers import Trainer
import torch

# Cross-version safe trainer with class weights
class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    # Accept extra kwargs (e.g., num_items_in_batch) for newer Transformers
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits  = outputs.get("logits")

        if self.class_weights is not None:
            loss_fct = torch.nn.CrossEntropyLoss(
                weight=self.class_weights.to(logits.device)
            )
        else:
            loss_fct = torch.nn.CrossEntropyLoss()

        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [7]:
import transformers, os
print("Transformers:", transformers.__version__)

# 1) Model (continue from initial)
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL_DIR,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

# 2) Training args — NO modern flags
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    logging_dir=os.path.join(OUTPUT_DIR, "logs"),
    logging_steps=50
)

# 3) Trainer (use the cross-version WeightedTrainer you added earlier)
trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,          # deprecation warning is fine
    compute_metrics=compute_metrics,
    class_weights=cw
)

# 4) Train → Evaluate → Save
trainer.train()
metrics = trainer.evaluate()
print("Val metrics:", metrics)

os.makedirs(MODEL_OUT, exist_ok=True)
trainer.save_model(MODEL_OUT)
tokenizer.save_pretrained(MODEL_OUT)
print("Saved to:", MODEL_OUT)


Transformers: 4.55.0


C:\Users\Hp\AppData\Local\Temp\ipykernel_57104\790932965.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
C:\hp\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,0.581100


C:\hp\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Val metrics: {'eval_loss': 0.5323199033737183, 'eval_accuracy': 0.8529411764705882, 'eval_f1': 0.8780487804878049, 'eval_precision': 0.8181818181818182, 'eval_recall': 0.9473684210526315, 'eval_runtime': 9.811, 'eval_samples_per_second': 6.931, 'eval_steps_per_second': 0.306, 'epoch': 4.0}
Saved to: ../outputs/with_pseudo\model


In [8]:
os.makedirs(MODEL_OUT, exist_ok=True)
trainer.save_model(MODEL_OUT)
tokenizer.save_pretrained(MODEL_OUT)
print("Saved to:", MODEL_OUT)

from transformers import pipeline
clf = pipeline("text-classification", model=MODEL_OUT, tokenizer=MODEL_OUT, device=0 if torch.cuda.is_available() else -1)

examples = [
    "તને અહીં જીવવા ન દઈએ",         # potentially hateful
    "શું હાલ છે? બરાબર?",           # benign
    "ચુપ રહે, ગંદી ભાષા નો ઉપયોગ ન કર", # abusive/hate-ish
]
print(clf(examples))


Saved to: ../outputs/with_pseudo\model


Device set to use cpu


[{'label': 'non_hate', 'score': 0.5640935301780701}, {'label': 'non_hate', 'score': 0.564657986164093}, {'label': 'non_hate', 'score': 0.5645349025726318}]
